## Creating the network structure

In [1]:
#Packages
import numpy as np
import networkx as nx
from matplotlib import pyplot
import json


#Function that converts a graph in networkx format into a dictionary (nodes, first_neighbours)
def graph_to_dict(G):
    graph=dict()
    for i in G.edges:
        node_i = i[0]
        node_j = i[1]
        #Scanning over the edges and inserting the links in the dictionary
        if node_i not in graph:
            graph[node_i]=[node_j]
        else:
            graph[node_i].append(node_j)
        if node_j not in graph:
            graph[node_j]=[node_i]
        else:
            graph[node_j].append(node_i)        
    return graph

#Function that computes the degree distribution of a graph [input should be a dictionary]
def compute_degree_distribution(graph):
    dgr=np.zeros(max(max(graph.keys()),len(graph.keys())))
    for i in graph.keys():
        dgr[i-1]= len(graph[i])
    pyplot.hist(dgr,density=True)

#Function that prints the graph structure into a file [.json format] "filename"
def print_network_onfile(graph,filename):
    with open(filename, 'w+') as fp:
        json.dump(graph, fp)
        
#Function that loads the graph structure from the file "filename" into a dictionary
#with the format: [key:nodes,values:first_neighbours]
def load_network_fromfile(filename):
    json_data=open(filename).read()
    graph=json.loads(json_data)
    graph={int(k):[int(i) for i in v] for k,v in graph.items()}
    return(graph)


#<k> = 2*K/N -> Remember that the network should be fully connected for the case of ER random graphs
N = 10 # nodes
K = 40 # edges
m = 3    # number of stubs for the Barabasi-Albert random graph

#Creating an ER random graphs with N number of nodes and K number of edges
G = nx.gnm_random_graph(N, K)

#Creating a Barabasi-Albert 
#G=nx.barabasi_albert_graph(N,m) with m stubs on each step

#Creating the dictionary structure from the graph G
graph_structure = graph_to_dict(G)

#Computing the degree distribution from the dictionary structure of a graph
compute_degree_distribution(graph_structure)

#Printing the network on a file
#print_network_onfile(graph_structure)

#Loading the network as a dictionary structure from a file
#graph_structure=load_network_fromfile("result.json")

print(graph_structure)


{0: [6, 4, 8, 3, 2, 5, 1, 9, 7], 6: [0, 1, 2, 3, 4, 5, 9, 7], 4: [0, 1, 2, 3, 5, 8, 6, 7], 8: [0, 1, 2, 4, 5, 7, 9], 3: [0, 1, 5, 9, 7, 6, 4], 2: [0, 1, 8, 5, 4, 6, 7, 9], 5: [0, 1, 2, 3, 4, 6, 9, 7, 8], 1: [0, 5, 3, 6, 7, 8, 9, 2, 4], 9: [0, 1, 2, 3, 5, 6, 8], 7: [0, 1, 2, 3, 4, 5, 6, 8]}
